# OSTEP CH6: Mechanism: Limited Direct Execution


# 1. Virtualisation of CPU

## 1.1 Definition:
- OS run many jobs (processes) simultaneously
- Sharing one physical CPU (i.e. time sharing)

## 1.2 Challenges
- **Performance**: 
    - Implement without excessive overhead on system.
    - Efficiently
- **Control** (over processes): Otherwise 
    - processes can run forever
    - procesess take over machine
    - processes can access disallowed infromation

# 2. Virtualisation Techniques

# 2.1 Direct Execution
**How**: Run program natively on hardware CPU  

**Pro**: Fast  

**Con**: Problem 1: **Restriction Operations**  
- Process unable to perform **restricted operations** on CPU  
- How to perform **Restricted Operations**?  
    - I.e. I/O & other restricted operations
    - In controlled (process) manner 

# 2.2 Privledge Separation
- Disallowing untrusted programs having full control of our system
- Prevent memory corruption, crashing OS, or private data access

# 2.3 Processor Modes: `User` vs `Kernel`
- `User`: limited privledges
- `Kernel`: full privledges


# 3. CPU vs OS: Primer


## 3.1 Layers & Modes

|Layer|Provides|Used by|Co-evolution Notes|
|-----|--------|-------|------------------|
|**`OS`<br>(software)**|**uses** the ***mechanism***:<br> - to enforce control, protection & abstraction|User Programs|OS have kernel/user mode:<br> - *because* CPU **supports** it|
|**`CPU`<br>(hardware)**|**provides** the ***mechanism***:<br> - for **user** & **kernel** modes|OS kernel|Modern CPU's support it:<br> - *because* **OS's need it**|

In laymens:
- the `CPU` makes it ***possible***
- the `OS` makes it ***useful***

# 4. User Process & Privledged Operation
**Problem**: 
- What should a `user` **process** do when it wishes to perform a `priviledged` **operation**?

**Solution**: 
- Modern hardware provides user programs to perform a `system call`:
    - Allow kernel to carefully expose key functionality to user programs
    - E.g. accessing file system, creat/destroy processes, communicate other procs, allocate more memory.
    
**How to execute `system call`**:
- Program issue special `trap` instruction (trap into kernel, raise priviledge level to kernel mode, perform operations, return from trap)
- Hardware (when trapping) must ***save*** **caller's** `registers` to return correctly after OS issues return-from-trap.
    - x86: Processor push program counter, flags, other regs, onto a per-process `kernel stack`, ret-f-trap pops values off the stack and resume user program

**How does `Trap` know what code to run inside OS**
- `Kernel` must ***control*** what code executes upon trap by setting up `trap table` at boot time.


In [2]:
2**32

4294967296

| Component                            | Description                                                            |
| ------------------------------------ | ---------------------------------------------------------------------- |
| **Instruction Pointer (IP / RIP)**   | Where the process was executing (next instruction)                     |
| **Stack Pointer (SP / RSP)**         | Where the top of the stack is (for local vars, return addresses, etc.) |
| **Flags Register (EFLAGS/RFLAGS)**   | Condition bits, interrupt enable bit, etc.                             |
| **General-purpose registers**        | Like `RAX`, `RBX`, `RCX` — store intermediate values                   |
| **Segment Registers (CS, SS, etc.)** | Define memory protection/permissions and mode (user vs kernel)         |
| **Page Table Base (CR3)**            | Points to the memory map of the process                                |


| Component                 | Description                                              |
| ------------------------- | -------------------------------------------------------- |
| **Memory space**          | Code, heap, stack, and memory-mapped files               |
| **Open file descriptors** | List of files/sockets/devices the process can read/write |
| **Process ID (PID)**      | Unique ID                                                |
| **Parent process info**   | Used for process hierarchy and signaling                 |
| **Signal handlers**       | Custom handlers for `SIGINT`, `SIGSEGV`, etc.            |
| **Scheduling priority**   | Determines CPU time                                      |
| **Kernel stack**          | Used during kernel-mode execution                        |
| **User stack**            | For user-mode execution                                  |
| **Environment variables** | Passed at process creation                               |
| **Security credentials**  | UID, GID, capabilities, etc.                             |


        Process
        ├── CPU Context (key state)
        │   ├── Instruction Pointer
        │   ├── Stack Pointer
        │   ├── Registers
        │   └── Flags
        ├── Kernel Stack
        ├── User Stack
        ├── Heap
        ├── Code (.text)
        ├── Open Files
        ├── Signals
        ├── Identity (PID, UID)
        └── Scheduling Info
✅ Every process has a core execution context: registers, IP, SP, flags — saved/restored during context switches.

✅ On top of that, it has its own custom execution environment: memory, file descriptors, privileges, etc.

✅ The OS tracks all of this (usually in a task_struct in Linux).

✅ You can think of this “key state” as the "CPU-facing" identity of the process — everything else is process-facing or OS-facing.

**System Calls** 
- allow `kernel` to provide `user programs` 
- some `functionalities` , e.g.:
    - access file system
    - creating & destroying processes
    - communicating with other processes
    - allocating more memory


Implement CPU Virtualisatoin with
- limited direct execution (i.e. some key low-level mechanisms)
- steps:
    - run program from cpu
    - set limits on processes without OS assistance
Sections:
0. Virtualisation
- time sharing
- challenges:
    - performance
    - control

1. What is Limited Direct Execution (LDE):
- A basic virt technique
- Open program directly in CPU
    - **OS** actions overview
    - **Program** actions overview
- Whats needed to Virt CPU:
    - How to ensure program does what we want only, efficiently?
    - How OS implement time sharing, to switch processes, thurs virt the cpu?
    - what is the `limited` part?
        - limits on running programs?



2. LDE Problem 1: Restricted Operations
- DE 
    - pros: fast, runs native on hardware CPU
    - cons: problem whence process wants perform **restricted operation**, e.g.:
        - I/O request to disk
        - gain access more system resources: cpu, memory;
    - `crux`: how process to perform *i/o* & other *restricted operations* **without giving process complete control over system**
- `aside`: system call vs c-procedure call
- `user mode`:
- `kernel mode`:
- `protected control transfer`:
- hardware trapping mechanism:
    - `trap` special instruction:
    - `return-from-trap`:
    - `kernel stack` per-process:
    - `trap table` set up at boot time:
    - `trap handlers`:
- `LDE` protocol
    - two phases: 
        - boot time: trap table & CPU priviledged instruction
        - running process: 
            - start: node to plist, alloc_mem, retrun trap, start process, user mode
            - syscall: trap back os, os handles, return control return-from-trap to process
            - complete: process returns to main/stub code to exit properly
    - timeline: process, in/out kernel, registers (gen purpose reg, program counter), save to/fro hardware
- `system-call number`:
    - `level of indirection`:
- `priviledged` operation:


3. LDE Problem 2: Switching Between Processes
- `crux`: how OS **regain control** from CPU to switch processes
- `cooperative` approach: `Wait For Sys Calls`
    - `yield`
    - `trap`
- `non-cooperative` approach: `OS takes control`
    - `timer-interrupt`
    - `interrupt handler`
- `saving-restoring context`:
    - `scheduler`
    - `context switch`
    - registers:    
        - `user registers`
        - `kernel registers`
    - `LDE timer interrupt diagram`

4. LDE Concurrency
- `disable interrupts`
- `locking schemes`
- `xv6 context switch` code




Direct Execution Protocol (Wihtout Limits)
- Add entry to process list
- Allocate memory to program
- Load program to RAM
- Setup Stack: argc, argc
- Clear registers
- Execute main()


Direct Execution Protocol (With Limits)
- Add entry to process list
- Allocate memory to program
- Load program to RAM
- Setup Stack: argc, argc
- 
- Clear registers
- Execute main()

## 3.2 Privledge Separation Examples

## 3.2.1 Without Privledge Separation
Early CPU's or toy OS: 
- allows programs 
- issue hardware commands
- directly to disk controller

## 3.2.2 With `user`/`kernel` mode 
Modern CPUs & OSes
- User program calls `write()`
    - OS intercepts via `syscall`
    - Switch to `kernel` mode
    - OS checks: Do you have permission? file open? Write quota?
    - If yes, OS issues disk command
- Results back to user program

In C, function arguments are passed by value , meaning functions get copies of what you pass. So if you want to modify a variable inside a function, you must pass a pointer  to it.

`printf()` - format specifier
- `%zu` for `sizeof(nums[5])`;


- `typedef char String[50]`: 50 bytes for each String (nickname for char)
- `typedef char* String`: Pointer, no need to allocated bytes

Set array of chars with limits
- `#define NO_OF_NAMES 3`
- `#define MAX_NAMES_LEN 50`
- `char names[NO_OF_NAMES][MAX_NAMES_LEN]={0};`


pointers and memory addresses:

- `int age     = 69;`
- `int *pAge   = &age;`
- `int* p2Age  = &age;`
- `printf("%d is at: %p\n",age,&age);`
- `printf("*pAge is at:  %p\n",pAge);`
- `printf("*p2Age is at: %p\n",p2Age);`
- [gpt] `pChar[0]` is shorthand for `*(pChar + 0)`, which is just `*pChar`.

# writing to file
- Goal: Create a ***pointer*** to a `FILE` struct
    - `struct` is **container/blueprint** for some real world representation
    - `FILE` is **data_type** of `struct` from `stdio.h`
    - `FILE *pFile`: pointer to file???
    - `fopen()`: (attempts) to create a file
        - arg1: file path, abs/rel.
        - arg2: `r` or `w` mode
        - returns: 
            - IF FAILED: `NULL` (addres 0 or pointer to nothing?)
            - IF SUCESS: ...    
    - `fclose(ptr_file_struct)`

- [1] Writing text: 
- `char[]="aksdjfkl\nlaskdjflaksdf"`

- `fprintf()`: write to file
    - arg1: pointer to file
    - arg2: format specifier
    - arg3: variable
    - return: >0 success, or <0 failure



# read file
- define a pointer to where the file is located
- create char buffer
- `fgets()` return NULL when exhausted
    - arg1: character buffer
    - arg2: size of buffer
    - arg3: file description or pointer to file

- while `fgets!=NULL`...

# ERROR stuff
- `perror("...")`: prints error in buffer?
- `stderr`: error stream
- `errorno`: special global variable that's automatically set by system calls and library functions when they encounter errors
- `stderr(errorno)`: get the error msg?
        fprintf(stderr, "Error opening file: %s ❌\n", strerror(errno));

#### code examples (inside `pFile=NULL`):
- `perror("error msg:")`
- `fprintf(stderr, "error msg: %s\n", strerror(errno));`

# Dynamic Memory `malloc()`
- `malloc(n_bytes)` returns a **pointer** ---> to array-like data-structure
    - arg1: `n_bytes` is number of bytes to reserve in memory
    - calculate `n_bytes` at run-time: 
        - e.g. Ask user how **many elements**, then **multiple** by `size of char` (as they're entering chars?)

- **Memory**: from the `heap`, usually its from the `stack` 